In [32]:
import pandas as pd
from lexrank import LexRank
from lexrank.mappings.stopwords import STOPWORDS
from path import Path
import os
import re

# LexRank Test File

Following along with the [LexRank Tutorial](https://github.com/crabcamp/lexrank).

In [3]:
docs = []
doc_dir_tng = Path('../StarTrekNextGenScriptData/scripts_tng')
doc_dir_dsn = Path('../StarTrekNextGenScriptData/scripts_ds9')
doc_dirs = [doc_dir_dsn,doc_dir_tng]

# open sentences too
complete = pd.read_csv('../StarTrekNextGenScriptData/complete_data.csv')

In [4]:
# using the ENTIRE script, which may not have useful data (lots of empty lines)
for documents_dir in doc_dirs:
    for file_path in documents_dir.files('*.txt'):
        with file_path.open(mode='rt', encoding='latin-1') as fp:
            docs.append(fp.readlines())

In [ ]:
short_docs = docs[:50]

In [ ]:
lxr = LexRank(docs, stopwords=STOPWORDS['en'])
# takes about 1:48

In [12]:
sentences = complete[complete.file=='504.txt']['quote']

In [30]:
summary = lxr.get_summary(sentences, summary_size=5, threshold=.3)
summary

['   You know your wife well, Miles.    But she knows you even better.',
 '   Miles?',
 '   Miles!',
 "   You've done well, Miles.",
 "   We've gone over this, Miles. It's    not your concern."]

In [31]:
' '.join(['hey','hjkhdl'])

'hey hjkhdl'

In [44]:
def get_summary_per_episode(file_name:str,df:pd.DataFrame,lxr:LexRank,summary_len:int=5):
    '''get the sentence summary per episode'''
    sentences = list(df[df.file==file_name]['quote'])
    summary = lxr.get_summary(sentences, summary_size=summary_len,threshold=.3)
    output = '...\n'.join(summary)
    output= re.sub(r'\s+', ' ', output.strip())
    return output

In [46]:
get_summary_per_episode('508.txt',complete,lxr)

'The Captain\'s not going to die. He\'s the Emissary. The Prophets will take care of him.... Are you speaking as a Starfleet Captain or as the Emissary of the Prophets?... So in the meantime, he\'s going to keep having... "visions." I knew we were headed for trouble the minute he allowed the Bajorans to call him "Emissary."... What is it, Emissary? Have the Prophets revealed something to you?... May the Prophets reveal their wisdom to you, Emissary.'

In [54]:
complete[complete.file=='410.txt']['series'].unique()[0]

'Deep Space Nine'

In [57]:
episodes = complete['file'].unique()
output = []
for file in episodes:
    ep = complete[complete.file==file]['episode'].unique()[0]
    series = complete[complete.file==file]['series'].unique()[0]
    summary = get_summary_per_episode(file,complete,lxr)
    output.append([ep,series,file,summary])

summary_df = pd.DataFrame(output, columns=['episode','series','file','summary'])
# takes about a minute 30

In [69]:
summary_df.sample(6)

,episode,series,file,summary
55,Move Along Home,Deep Space Nine,410.txt,Allamaraine... Third shap... Ferengi lagunda.....
295,"Time's Arrow, Part II",The Next Generation,227.txt,The triolic levels are as high as they were on...
45,Battle Lines,Deep Space Nine,413.txt,Only three of us will be coming back... Kai Op...
294,The Wounded,The Next Generation,186.txt,"We've confirmed the report, Captain. It was th..."
106,Wrongs Darker Than Death or Night,Deep Space Nine,541.txt,"Computer. Locate Cardassian occupation files, ..."
104,The Homecoming,Deep Space Nine,421.txt,It's good to know that Bajor is free.... Chief...


In [71]:
summary_df[summary_df.episode=="The Wounded"]['summary'].values[0]

"We've confirmed the report, Captain. It was the starship Phoenix... under the command of Benjamin Maxwell.... We are not going after any more Cardassian ships, Captain Maxwell... you are going to return to your bridge and set a course for Starbase Two-one-one... the Phoenix and the Enterprise will return to Federation space together. I will permit you the dignity of retaining your command during our voyage. My only alternative would be to put you in our brig, while we tow your ship back to starbase in disgrace.... Picard here.... Oh, one more thing, Macet. Maxwell was right. Those ships weren't carrying scientific equipment, were they?... Captain Picard is waiting in his Ready Room. This way."

My thoughts here:

We can make a way to search for episodes based on the episode summary!